In [ ]:
import networkx as nx
import random
import matplotlib.pyplot as plt

### Part One
- Defining a network topology
- Randomly assigning colours to each node 
- Running experiments to see if correct colouring of the graph can be reached.
- Counting the number of conflicts over time

In [ ]:
n = 60
k = 32
p = 0.7

G = nx.random_graphs.connected_watts_strogatz_graph(n, k, p)


In [ ]:
nx.draw(G)

In [ ]:
A = nx.adjacency_matrix(G)
print(A.todense())

In [ ]:
def random_colouring(G, colours):
    for n in G:
        G.nodes[n]["colour"] = random.randrange(0, colours)

In [ ]:
collisions_c = []

colours = []

for i in range(3, 12):
    colours.append({'num': i, 'max_collisions': 0, 'min_collisions': 50})

def check_colouring(G, num_colours, collision, max_collisions, min_collisions):
    for node in G:
        for n in G.neighbors(node):
            if G.nodes[node]['colour'] == G.nodes[n]['colour'] and node > n:
                G.nodes[node]['colour'] = random.randint(0, num_colours-1)
                collision = collision + 1


    if collision > max_collisions:
        max_collisions = collision

    if collision < min_collisions:
        min_collisions = collision
    
    return collision, max_collisions, min_collisions

for i in range(len(colours)):
    collisions_c.append([])
    random_colouring(G, colours[i]['num'])
    for j in range(10):
        collisions_c[i].append(0)
        collisions_c[i][j], colours[i]['max_collisions'], colours[i]['min_collisions'] = check_colouring(G, colours[i]['num'], collisions_c[i][j], colours[i]['max_collisions'], colours[i]['min_collisions'])
    print(f"Collisions over time for {colours[i]['num']} colours: {collisions_c[i]}")
    print(f"Max collisions: {colours[i]['max_collisions']} for number of colours: {colours[i]['num']}")
    print(f"Min collisions: {colours[i]['min_collisions']} for number of colours: {colours[i]['num']}")


In [ ]:
import csv

with open('collisions.csv', mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Number of colours', 'Collisions over time', 'Max collisions', 'Min collisions'])
    for i in range(len(colours)):
        writer.writerow([colours[i]['num'], collisions_c[i], colours[i]['max_collisions'], colours[i]['min_collisions']])

### Part Two

In [ ]:
# Produce correct colouring by removing edges from the graph
# - find nodes "in conflict"
# - remove the edge

pruned_graph = G.copy()

# Recolour a graph with a number of colours
# Prune edges that cause unresolvable collisions
def prune_graph(G, colours):
    # Sort the nodes based on degree; low degree nodes can be recoloured easily
    nodes = sorted(list(G), key=lambda x: G.degree[x], reverse=True)
    #nodes = list(G)
    print(nodes)
    # Iterate through nodes in the graph
    for n in nodes:
        # Set conflicts to empty, set all colours to available
        conflicts = []
        c_available = dict([(c, True) for c in list(range(colours))])
        # Iterate through node's neighbours
        for m in G.neighbors(n):
            # Mark neighbour colour as unavailable; append to conflicts if it shares current node colour
            c_available[G.nodes[m]["colour"]] = False
            if G.nodes[n]["colour"] == G.nodes[m]["colour"]:
                conflicts.append((n, m))
        # Resolve existing conflicts
        if conflicts != []:
            solved = False
            # Attempt to recolour
            for c in c_available:
                if c_available[c]:
                    G.nodes[n]["colour"] = c
                    solved = True
            # Remove conflicting edges if recolour is not possible
            if not solved:
                G.remove_edges_from(conflicts)

c = 2
random_colouring(pruned_graph, c)

s_pre_p = pruned_graph.size()
d_pre_p = nx.diameter(pruned_graph)
print(f"Before Pruning:\n\tSize:\t\t{s_pre_p}\n\tDiameter:\t{d_pre_p}")
prune_graph(pruned_graph, c)
s_post_p = pruned_graph.size()
d_post_p = nx.diameter(pruned_graph)
print(f"After Pruning:\n\tSize:\t\t{s_post_p}\n\tDiameter:\t{d_post_p}")
print(f"Graph is {float(s_post_p) / float(s_pre_p) * 100}% of its original size\nDiameter increased by {((float(d_post_p) / float(d_pre_p)) - 1) * 100}%")
pos = nx.bipartite_layout(pruned_graph, [n for n in pruned_graph if pruned_graph.nodes[n]["colour"] == 0])
nx.draw(pruned_graph, node_color=[pruned_graph.nodes[n]["colour"] for n in pruned_graph], pos=pos, node_size=10)